In [1]:
import pandas as pd
import numpy  as np
from pprint import pprint as pp

# Data

Categorical data를 다룰때 2가지로 분류할수 있다. Nominal 또는 Ordinal. 

* **color**는 nominal data로서 순서가 없다. 
* **size**는 ordinal data로서 순서를 갖을수 있다. 
* **labal**은 supervised learning에서 사용되는 target으로서, MNIST로 따지면.. 숫자 정답이라고 보면 됨

In [2]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'], 
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']
])
df.columns = ['color', 'size', 'price', 'label']
df

,color,size,price,label
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


# Mapping Ordinal Features

In [3]:
# Do not run this code more than 1 
size_mapping = {
    'M': 1,
    'L': 2,
    'XL': 3}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,label
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


# Encoding Class Lables

Scikit-Learn 내부적으로 class lables(target)을 다룰때 intergers값으로 변환시켜서 처리하지만.. <br>
technical glitch를 줄이기 위해서 집접 변환시켜주는게 좋습니다. 

class labels은 어떤 순서대로 놓을수 없는 nominal data이기 때문에, 어떤 label에 특정 값을 mapping시켜줘야하는지는 중요하지 않습니다.

In [4]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['label']))}
print('Class Labels:', class_mapping)

# Convert class labels into integers
df['label'] = df['label'].map(class_mapping)
df

Class Labels: {'class1': 0, 'class2': 1}


,color,size,price,label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


### Reverse the mapping
다시 String으로 변환시키는것은 다음과 같이 합니다.

In [5]:
class_mapping_inv = {v: k for k, v in class_mapping.items()}
df['label'] = df['label'].map(class_mapping_inv)
df

,color,size,price,label
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


### encoding class labels with Scikit-Learn

In [6]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df['label'] = class_le.fit_transform(df['label'].values)
df

,color,size,price,label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


다시 Inverse 시키는것은 다음과 같이 합니다.

In [7]:
class_le.inverse_transform(df['label'].values)

array(['class1', 'class2', 'class1'], dtype=object)

# One-hot encoding

In [8]:
X = df[['color', 'size', 'price']].values
print('[Data]')
print(X)

color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
print('\n\n[Transformed Data]')
print(X)

[Data]
[['green' 1 10.1]
 ['red' 2 13.5]
 ['blue' 3 15.3]]


[Transformed Data]
[[1 1 10.1]
 [2 2 13.5]
 [0 3 15.3]]


색상들을 모두 integers값으로 변환을 시켰습니다.<br>

* blue -> 0
* green -> 1
* red -> 2

<span style="color:#ee3333">
만약 여기에서 멈추고, 분류를 한다면 categorical data를 다루는 것중 가장 흔한 실수중에 하나를 저지르는 것입니다.<br>
색상은 특정 순서를 따르지 않습니다. 만약 저렇게 0, 1, 2로 분류를 한다면 blue보다 green이 더 크고, green보다 red가 더 크다는 잘못된 뜻이 될 수 있습니다.<br>
이런 경우 one-hot vector로 변경해주어야 합니다.
</span>

In [9]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0]) # 어떤 columns을 one-hot vector로 만들지 정함
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

ohe.fit_transform(X) 를 실행시키면 sparse matrix를 return합니다. <br>
sparse matrix는 더 효율적이지만, visualization을 못합니다.<br>
따라서 Numpy Array로 변형시켜줘야 합니다. 

방법은 toarray()를 사용하거나 (..., sparse=False) 옵션을 주면 됩니다.

In [10]:
ohe = OneHotEncoder(categorical_features=[0], sparse=False)
ohe.fit_transform(X)

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])